In [4]:
from src.api.api_helper import get_popular_topics
from src.api.api_helper import setup_session
from src.api.api_helper import get_entries_from_topic
from src.model.eksi import ProcessStatus
from src.model.db_models import TopicData
import json
import time

HOST = "https://api.----.com"
host =  "https://api.--------.com"

In [5]:
session = setup_session()

In [6]:
remaining_topics = []

with open('topics.json') as tempfile:
    for line in tempfile:
        remaining_topics.append(line.strip())

In [ ]:
all_authors  = set()
all_entries = []
all_topics = []
ex_count = 0

for index in reversed(range(6850009,6880019)):
    if index % 300 == 0:
        with open('temp_entries.txt', 'a', encoding="utf-8") as file_entries:
            for entry in all_entries:
                file_entries.write(json.dumps(entry.dict(), ensure_ascii=False) + "\n")

        with open('temp_authors.txt', 'a', encoding="utf-8") as file_authors:
            for author in all_authors:
                file_authors.write(json.dumps(author.dict(), ensure_ascii=False) + "\n")

        with open('temp_topics.txt', 'a', encoding="utf-8") as file_topics:
            for topics in all_topics:
                file_topics.write(json.dumps(topics.dict(), ensure_ascii=False) + "\n")

        all_authors  = set()
        all_entries = []
        all_topics = []
        ex_count = 0

        print(index, "saved")
    
    try:
        entries, topic = get_entries_from_topic(session=session, topic_id=index, page=1, entries=[])

        for entry in entries:

            # add author
            author = entry.Author
            author.ProcessStatus = ProcessStatus.NOT_PROCESSED.value
            all_authors.add(author)
            
            entry.TopicId = topic.Data.Id
            entry.TopicTitle = topic.Data.Title
            all_entries.append(entry)

        if topic and topic.Success:
            created_at = ""
            if len(topic.Data.Entries) > 0:
                created_at = topic.Data.Entries[0].Created

            all_topics.append(TopicData(
                Id=topic.Data.Id,
                Title=topic.Data.Title, 
                EntryCounts=topic.Data.EntryCounts.Total,
                Slug=topic.Data.Slug,
                Created=created_at))
    except Exception as ex:
        ex_count += 1
        print(index, ex)
        if ex_count > 20001:
            break
            
with open('temp_entries.txt', 'a', encoding="utf-8") as file_entries:
    for entry in all_entries:
        file_entries.write(json.dumps(entry.dict(), ensure_ascii=False) + "\n")

with open('temp_authors.txt', 'a', encoding="utf-8") as file_authors:
    for author in all_authors:
        file_authors.write(json.dumps(author.dict(), ensure_ascii=False) + "\n")

with open('temp_topics.txt', 'a', encoding="utf-8") as file_topics:
    for topics in all_topics:
        file_topics.write(json.dumps(topics.dict(), ensure_ascii=False) + "\n")